In [294]:
import sys
from bert import QA
import gzip
from tqdm import tqdm
import json
# import spacy
# from spacy.symbols import ORTH
# import en_core_web_sm
# nlp = en_core_web_sm.load()
from fuzzywuzzy import fuzz
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)
from bs4 import BeautifulSoup

In [ ]:
# https://github.com/kamalkraj/BERT-SQuAD

In [3]:
model = QA('model')

doc = "Victoria has a written constitution enacted in 1975, but based on the 1855 colonial constitution, passed by the United Kingdom Parliament as the Victoria Constitution Act 1855, which establishes the Parliament as the state's law-making body for matters coming under state responsibility. The Victorian Constitution can be amended by the Parliament of Victoria, except for certain 'entrenched' provisions that require either an absolute majority in both houses, a three-fifths majority in both houses, or the approval of the Victorian people in a referendum, depending on the provision."

q = 'When did Victoria enact its constitution?'

answer = model.predict(doc,q)


In [4]:
answer['answer']

'1975'

In [1]:
import gzip
from tqdm import tqdm
import json
i = 0
list_of_q = []
# with gzip.open('../tiny-dev/nq-dev-sample.jsonl.gz') as f:
with gzip.open('v1.0-simplified_nq-dev-all.jsonl.gz') as f:
    for line in tqdm(f):
        list_of_q.append(line)
        i += 1

7830it [00:33, 234.70it/s]


In [29]:
import json
l = list_of_q[20]
x = json.loads(l)

In [32]:
x.keys()

dict_keys(['annotations', 'document_html', 'document_title', 'document_tokens', 'document_url', 'example_id', 'long_answer_candidates', 'question_text', 'question_tokens'])

In [30]:
x["document_url"][x["document_url"].index("title")+6:x["document_url"].index("&amp")]

'World_Economic_Forum'

In [42]:
from datetime import date
str(date.today())

import pandas as pd

In [45]:
ret = ["", 3]
i = "happy"
if i not in ret[0]:
    ret[0] = ret[0] + i + " "
ret.insert(0, 4)
ret

[4, 'happy ', 3]

In [ ]:
rows = []
for q in list_of_q:
    row, cols = get_all_revs(q)
    rows.append(row)
    
scoress = pd.DataFrame(rows, columns = cols).sort_values(['Overall Avg'], ascending = False)
scoress
# get "correct" one too 

In [ ]:
def get_all_revs(question):
    compares = {}
    x = json.loads(question)
    sa = get_short_answers(x)
    qtext = x["question_text"]
    title = x["document_url"][x["document_url"].index("title")+6:x["document_url"].index("&amp")]
    dates = [date.fromisoformat('2013-01-01'), date.fromisoformat('2017-01-01'), date.today()]
    ret = ["", qtext]
    col_names = ["GT Answers", "Question"]
    all_avg = 0
    for date in dates:
        doctext, _ = get_wikipedia_revision(title, date, sa)
        avg = 0
        ans = "No Edit"
        if doctext not None:
            avg, sets, sort, _, ans, pred, _ = compare(qtext, doctext) #['Average Score', 'Set Score', 'Sort Score','Question', 'Answer', 'Predicted', 'Length of Text'])
            if pred not in ret[0]:
                ret[0] = ret[0] + pred + " "
        ret.append(avg, ans)
        all_avg += avg
        col_names.append("Avg Score: " + str(date))
        col_names.append("Answer: " + str(date))
        
    ret.insert(0, all_avg/3)
    col_names.insert(0, "Overall Avg")
    return vals, col_names


def compare(qtext, doctext, short_answers):

    question_text = qtext
    doc = doctext
    q = question_text
    answer = model.predict(doc,q)
    if display:
        print('Predicted Answer: ', answer['answer'])
    
    set_scores = []
    sort_scores = []
    all_avg_scores = []
    for sa in short_answers:
        token_set_ratio = fuzz.token_set_ratio(sa, answer['answer'])
        token_sort_ratio = fuzz.token_sort_ratio(sa, answer['answer'])
        avg_score = (token_set_ratio + token_sort_ratio) / 2
        if display:
            print('Correct Answer: ', sa)
            print('Fuzzy score: ', avg_score)
        set_scores.append(token_set_ratio)
        sort_scores.append(token_sort_ratio)
        all_avg_scores.append(avg_score)
    max_index = all_avg_scores.index(max(all_avg_scores))
    return (max(all_avg_scores), max(set_scores), max(sort_scores), question_text, list(short_answers)[max_index], answer['answer'], len(document_text))

In [31]:
x["document_url"]

'https://en.wikipedia.org//w/index.php?title=World_Economic_Forum&amp;oldid=833924980'

In [265]:
def get_answers(question):
    answers = []
    for c in question['long_answer_candidates']:
        start_token = c['start_token']
        end_token = c['end_token']
        print(start_token, end_token)
        a = " ".join(t['token'] for t in question['document_tokens'][start_token:end_token]if not t['html_token'])
        answers.append(a)
    return answers


def get_tokens(question):
    answers = []
    for c in question['long_answer_candidates']:
        start_token = c['start_token']
        end_token = c['end_token']
        answers.append((start_token, end_token, len(question['document_tokens'])))
    return answers   



def get_short_answers(question):
    answers = []
    for annotation in question['annotations']:
        if len(annotation['short_answers']) != 0:
            short_answer = annotation['short_answers'][0]
#             print(short_answer)
            start_token = short_answer['start_token']
            end_token = short_answer['end_token']
#             print(question['document_tokens'][start_token:end_token])
            a = " ".join(t['token'] for t in question['document_tokens'][start_token:end_token])
            answers.append(a)
#         else:
#             print('No short answers found in annotation.')
            
    return set(answers)

In [266]:
print('Question: ' , x['question_text'])
# print(x.keys())
# print(x['long_answer_candidates'][0])
# print(get_answers(x))
short_answers = get_short_answers(x)
short_answers

Question:  what’s the difference between downhill and super g


set()

In [267]:
document_text = " ".join(t['token'] for t in x['document_tokens'])


In [268]:
question_text = x['question_text']
question_text

'what’s the difference between downhill and super g'

In [269]:
doc = document_text
q = question_text

answer = model.predict(doc,q)

In [270]:
answer['answer']

'super-G course consists of widely set gates that racers must pass through'

In [272]:
# check how many have no short answers
ind = []
for i in range(len(list_of_q)):
    l = list_of_q[i]
    x = json.loads(l)
    short_ans = get_short_answers(x)
    if len(short_ans) != 0:
        ind.append(i)


In [273]:
len(ind)

4289

In [274]:
def get_prediction_and_question(question, display = False, parse_only_paragraphs = False):
    x = json.loads(question)
    if display:
        print('Question: ' , x['question_text'])
    short_answers = get_short_answers(x)
    if parse_only_paragraphs:
        document_html = " ".join(t['token'] for t in x['document_tokens'])
        soup = BeautifulSoup(document_html)
#         document_text = " ".join(t.text for t in soup.find_all('p'))
        document_text = " ".join(t.text for t in soup.find_all('p')[:25])
    else:
        document_text = " ".join(t['token'] for t in x['document_tokens'] if not t['html_token'])
    question_text = x['question_text']
    doc = document_text
    q = question_text
    answer = model.predict(doc,q)
    if display:
        print('Predicted Answer: ', answer['answer'])
    
    set_scores = []
    sort_scores = []
    all_avg_scores = []
    for sa in short_answers:
        token_set_ratio = fuzz.token_set_ratio(sa, answer['answer'])
        token_sort_ratio = fuzz.token_sort_ratio(sa, answer['answer'])
        avg_score = (token_set_ratio + token_sort_ratio) / 2
        if display:
            print('Correct Answer: ', sa)
            print('Fuzzy score: ', avg_score)
        set_scores.append(token_set_ratio)
        sort_scores.append(token_sort_ratio)
        all_avg_scores.append(avg_score)
    max_index = all_avg_scores.index(max(all_avg_scores))
    return (max(all_avg_scores), max(set_scores), max(sort_scores), question_text, list(short_answers)[max_index], answer['answer'], len(document_text))

In [275]:
all_results = []
for i in tqdm(ind):
#     tup = get_prediction_and_question(list_of_q[i], display = False)
    tup = get_prediction_and_question(list_of_q[i], display = False, parse_only_paragraphs = True)
    all_results.append(tup)

100%|██████████| 4289/4289 [39:29<00:00,  1.81it/s]  


In [276]:
tup = get_prediction_and_question(list_of_q[ind[3]], display = True, parse_only_paragraphs = True)


Question:  star wars the clone wars anakin voice actor
Predicted Answer:  voice of Anakin Skywalker in Star Wars : The Clone Wars and Star Wars Rebels
Correct Answer:  Matthew MacKendree `` Matt '' Lanter
Fuzzy score:  27.0


In [277]:
all_results = sorted(all_results, key=lambda x: x[0])

In [288]:
scores = pd.DataFrame(all_results, columns = ['Average Score', 'Set Score', 'Sort Score','Question', 'Answer', 'Predicted', 'Length of Text'])

In [292]:
scores = scores.sort_values(['Average Score'], ascending = False)

In [295]:
display(scores)

,Average Score,Set Score,Sort Score,Question,Answer,Predicted,Length of Text
4288,100.0,100,100,who sang smoke gets in your eyes first,Tamara Drasin,Tamara Drasin,11686
3836,100.0,100,100,what is the name of the gap between two front teeth,diastema,diastema,3821
3844,100.0,100,100,when was harry potter and the philosophers stone published,1997,1997,16923
3843,100.0,100,100,what is the word for clarified butter in the balkans and middle east,smen,smen,13311
3842,100.0,100,100,who is the director of nightmare before christmas,Henry Selick,Henry Selick,16159
...,...,...,...,...,...,...,...
144,0.0,0,0,when did the flash first appear on arrow,in the eighth episode of Arrow 's second season,2014,15079
143,0.0,0,0,who played young monica in love and basketball,Kyla Pratt,.,15052
142,0.0,0,0,when do you stop being a junior doctor,"it culminates in a post as a Consultant , a General Practitioner ( GP ) , or some other non-training post , such as a Staff grade or Associate Specialist post",.,10071
141,0.0,0,0,who was the first british team to win the european cup,Celtic,da,12801


In [287]:
len(scores[scores['Average Score'] >= 70])

1374

In [ ]:
def get_prediction_and_question(question, display = False, parse_only_paragraphs = False):
    x = json.loads(question)
    if display:
        print('Question: ' , x['question_text'])
    short_answers = get_short_answers(x)
    if parse_only_paragraphs:
        document_html = " ".join(t['token'] for t in x['document_tokens'])
        soup = BeautifulSoup(document_html)
#         document_text = " ".join(t.text for t in soup.find_all('p'))
        document_text = " ".join(t.text for t in soup.find_all('p')[:25])
    else:
        document_text = " ".join(t['token'] for t in x['document_tokens'] if not t['html_token'])
    question_text = x['question_text']
    doc = document_text
    q = question_text
    answer = model.predict(doc,q)
    if display:
        print('Predicted Answer: ', answer['answer'])
    
    set_scores = []
    sort_scores = []
    all_avg_scores = []
    for sa in short_answers:
        token_set_ratio = fuzz.token_set_ratio(sa, answer['answer'])
        token_sort_ratio = fuzz.token_sort_ratio(sa, answer['answer'])
        avg_score = (token_set_ratio + token_sort_ratio) / 2
        if display:
            print('Correct Answer: ', sa)
            print('Fuzzy score: ', avg_score)
        set_scores.append(token_set_ratio)
        sort_scores.append(token_sort_ratio)
        all_avg_scores.append(avg_score)
    max_index = all_avg_scores.index(max(all_avg_scores))
    return (max(all_avg_scores), max(set_scores), max(sort_scores), question_text, list(short_answers)[max_index], answer['answer'], len(document_text))